In [ ]:
!pip3 install fastapi pandas torch transformers uvicorn google-cloud-storage scikit-learn


In [ ]:
from fastapi import FastAPI, UploadFile, HTTPException
import pandas as pd
import torch
import json
import io
from transformers import BartTokenizer



import logging


In [ ]:
# Load the new JSON files and inspect their structure

with open("../datasets/saved_data/input_embeddings_12.json", 'r') as file:
    input_embeddings_data = json.load(file)

with open("../datasets/input_old_model/input_mask_invert_12.json", 'r') as file:
    input_mask_invert_data = json.load(file)

with open("../datasets/input_old_model/input_masks_12.json", 'r') as file:
    input_masks_data = json.load(file)

input_embeddings_data, input_mask_invert_data, input_masks_data

In [ ]:
import json

with open('path_to_your_embedding_file.json', 'r') as file:
    embeddings = json.load(file)

import numpy as np

embedding_matrix = np.array(embeddings)

import torch

embedding_tensor = torch.from_numpy(embedding_matrix)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
import math
import numpy as np

""" main architecture for open vocabulary EEG-To-Text decoding"""
class BrainTranslator(nn.Module):
    def __init__(self, pretrained_layers, in_feature = 840, decoder_embedding_size = 1024, additional_encoder_nhead=8, additional_encoder_dim_feedforward = 2048):
        super(BrainTranslator, self).__init__()
        
        self.pretrained = pretrained_layers
        # additional transformer encoder, following BART paper about 
        self.additional_encoder_layer = nn.TransformerEncoderLayer(d_model=in_feature, nhead=additional_encoder_nhead,  dim_feedforward = additional_encoder_dim_feedforward, batch_first=True)
        self.additional_encoder = nn.TransformerEncoder(self.additional_encoder_layer, num_layers=6)
        
        # print('[INFO]adding positional embedding')
        # self.positional_embedding = PositionalEncoding(in_feature)

        self.fc1 = nn.Linear(in_feature, decoder_embedding_size)

    def forward(self, input_embeddings_batch, input_masks_batch, input_masks_invert, target_ids_batch_converted):
        """input_embeddings_batch: batch_size*Seq_len*840"""
        """input_mask: 1 is not masked, 0 is masked"""
        """input_masks_invert: 1 is masked, 0 is not masked"""
        
        # input_embeddings_batch = self.positional_embedding(input_embeddings_batch) 

        # use src_key_padding_masks
        encoded_embedding = self.additional_encoder(input_embeddings_batch, src_key_padding_mask = input_masks_invert) 
        
        # encoded_embedding = self.additional_encoder(input_embeddings_batch) 
        encoded_embedding = F.relu(self.fc1(encoded_embedding))
        out = self.pretrained(inputs_embeds = encoded_embedding, attention_mask = input_masks_batch, return_dict = True, labels = target_ids_batch_converted)                    
        
        return out


In [ ]:
from transformers import BartForConditionalGeneration

_MODEL = None

def get_model():
    global _MODEL

    if _MODEL is None:
        pretrained_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
        checkpoint_path = '/Users/michaelholborn/Documents/SoftwareLocal/monotropism/thoughtx/local_checkpoint/us-central1_eeg-test_dodadqada_eeg-text_eeg-text_checkpoints_decoding_best_task1_task2_taskNRv2_finetune_BrainTranslator_skipstep1_b32_20_30_5e-05_5e-07_unique_sent.pt'  # Change to the path of your model
        _MODEL = BrainTranslator(pretrained_bart)
        model_weights = torch.load(checkpoint_path, map_location=torch.device('cpu'))
        _MODEL.load_state_dict(model_weights)
        _MODEL.eval()

    return _MODEL


In [ ]:
import torch

# Convert loaded data to PyTorch tensors and add batch dimension
input_embeddings_tensor = torch.tensor(input_embeddings_data).unsqueeze(0)
input_masks_tensor = torch.tensor(input_masks_data).unsqueeze(0).squeeze(-1)
input_mask_invert_tensor = torch.tensor(input_mask_invert_data).unsqueeze(0).squeeze(-1)



# # Convert loaded data to PyTorch tensors
# input_embeddings_tensor = torch.tensor(input_embeddings_data)
# input_masks_tensor = torch.tensor(input_masks_data)
# input_mask_invert_tensor = torch.tensor(input_mask_invert_data)

input_embeddings_tensor.shape, input_masks_tensor.shape, input_mask_invert_tensor.shape

In [ ]:
model=get_model()

results = []

pred_tokens_list = []
pred_string_list = []

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
placeholder_token = tokenizer("<s>", return_tensors="pt")

In [ ]:
with torch.no_grad():
        try:
            outputs = model(input_embeddings_tensor, input_masks_tensor, input_mask_invert_tensor, placeholder_token["input_ids"])
            # Extract the generated token IDs from the model's outputs
            logits=outputs.logits
            probs = logits[0].softmax(dim = 1)
            values, predictions = probs.topk(1)
            predictions = torch.squeeze(predictions)
            predicted_string = tokenizer.decode(predictions).split('</s></s>')[0].replace('<s>','')
            predictions = predictions.tolist()
            truncated_prediction = []
            for t in predictions:
                if t != tokenizer.eos_token_id:
                    truncated_prediction.append(t)
                else:
                    break
            pred_tokens = tokenizer.convert_ids_to_tokens(truncated_prediction, skip_special_tokens = True)
            # print('predicted tokens:',pred_tokens)
            pred_tokens_list.append(pred_tokens)
            pred_string_list.append(predicted_string)
            print('predicted string:',predicted_string)
            # results.append(generated_text)
        except Exception as e:
            logging.error(f"Error during inference: {str(e)}")
            results.append(str(e))

In [ ]:
pred_string_list


In [ ]:
from fastapi import FastAPI, UploadFile, HTTPException
import pandas as pd
import torch
import json
import io
from transformers import BartTokenizer



import logging


In [ ]:
import json

with open('path_to_your_embedding_file.json', 'r') as file:
    embeddings = json.load(file)

import numpy as np

embedding_matrix = np.array(embeddings)

import torch

embedding_tensor = torch.from_numpy(embedding_matrix)

In [ ]:
from transformers import BartForConditionalGeneration

_MODEL = None

def get_model():
    global _MODEL

    if _MODEL is None:
        pretrained_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
        checkpoint_path = '/Users/michaelholborn/Documents/SoftwareLocal/monotropism/thoughtx/local_checkpoint/us-central1_eeg-test_dodadqada_eeg-text_eeg-text_checkpoints_decoding_best_task1_task2_taskNRv2_finetune_BrainTranslator_skipstep1_b32_20_30_5e-05_5e-07_unique_sent.pt'  # Change to the path of your model
        _MODEL = BrainTranslator(pretrained_bart)
        model_weights = torch.load(checkpoint_path, map_location=torch.device('cpu'))
        _MODEL.load_state_dict(model_weights)
        _MODEL.eval()

    return _MODEL


In [ ]:
model=get_model()

results = []

pred_tokens_list = []
pred_string_list = []

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
placeholder_token = tokenizer("<s>", return_tensors="pt")

In [ ]:
pred_string_list
